In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from datetime import datetime
import os
import time
import requests
from pga.return_pid import return_pid

In [3]:

for tourny_num in range(5000):
    time.sleep(1)
    
    try:
        url = 'http://www.espn.com/golf/leaderboard?tournamentId={}'.format(tourny_num)

        response = requests.get(url).text

        soup = bs(response, 'lxml')
        result_df = pd.DataFrame(columns = ['player_id', 'player_name', 'tournament_name', 'tournament_start_date', 
                                        'tournament_end_date', 'num_rounds_played', 'first_round_score',
                                        'second_round_score', 'third_round_score', 'fourth_round_score', 
                                        'total_score', 'score_vs_par', 'final_rank', 'earnings'])

        header = soup.find('div', {'class': 'tab-pane active'})
        tournament_name = header.find('h1').text

        date_str = header.find('div', {'class': 'date'}).text
        
        month_ls = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                   'October', 'November', 'December']
        
        month_count = 0
        for m in month_ls:
            if m in date_str:
                month_count += 1
                
        if month_count == 1:
            month = date_str.split(' ')[0]
            start_day = date_str.split(' ')[1]
            end_day = date_str.split(' ')[3].replace(',','')
            year = date_str.split(' ')[-1]
            
            start_date_str = '{}_{}_{}'.format(month, start_day, year)
            start_date = datetime.strptime(start_date_str, '%B_%d_%Y')

            end_date_str = '{}_{}_{}'.format(month, end_day, year)
            end_date = datetime.strptime(end_date_str, '%B_%d_%Y')
            
            
        elif month_count == 2:
            month_start = date_str.split(' ')[0]
            start_day = date_str.split(' ')[1]
            
            month_end = date_str.split(' ')[3]
            end_day = date_str.split(' ')[4].replace(',', '')
            year = date_str.split(' ')[-1]
            
            start_date_str = '{}_{}_{}'.format(month_start, start_day, year)
            start_date = datetime.strptime(start_date_str, '%B_%d_%Y')

            end_date_str = '{}_{}_{}'.format(month_end, end_day, year)
            end_date = datetime.strptime(end_date_str, '%B_%d_%Y')
        else:
            print('Error in date for tourny {}'.format(tourny_num))

        players = soup.findAll('tbody')

        for p, player in enumerate(players):
            try:
                cells = player.findAll('td')
                finish_pos = cells[0].text
                player_name = player.find('a', {'class': 'full-name'}).text
                score_to_par = player.find('td', {'class': 'relativeScore sm asc in post'}).text
                round_1_score = cells[6].text
                round_2_score = cells[7].text
                round_3_score = cells[8].text
                round_4_score = cells[9].text
                total_score = cells[10].text

                round_count = 0

                for rnd in [round_1_score, round_2_score, round_3_score, round_4_score]:
                    try:
                        int(rnd)
                        round_count += 1
                    except:
                        pass



                try:
                    player_id = asdreturn_pid(player_name)
                except:
                    player_id = np.nan

                #return numeric version for rank

                finish_pos = finish_pos.replace('T', '')

                if finish_pos == '-':
                    finish_pos = -999

                finish_pos = int(finish_pos)

                #Return numeric version of to par
                score_to_par = score_to_par.replace('+', '')

                if score_to_par == 'E':
                    score_to_par = 0

                if score_to_par == 'CUT' or score_to_par == 'Cut':
                    score_to_par = -999

                if score_to_par == 'WD':
                    score_to_par = -9999
                    finish_pos = -9999

                if score_to_par == 'DQ':
                    score_to_par = -99999
                    finish_pos = -99999


                score_to_par = int(score_to_par)

                result_df.loc[p] = [player_id, player_name, tournament_name, start_date, end_date, round_count,
                                   round_1_score, round_2_score, round_3_score, round_4_score, total_score, score_to_par,
                                   finish_pos, np.nan]
            except:
                pass

        last_finish_pos = result_df['final_rank'].max() + 1

        cut_df = result_df.loc[(result_df['final_rank'] == -999)]
        cut_df = cut_df.loc[cut_df['total_score'] != '--']
        cut_df['total_score'] = cut_df['total_score'].apply(lambda x: int(x))
        cut_df['final_rank'] = cut_df['total_score'].rank(method = 'dense')

        ranks = cut_df['final_rank'].unique()

        rank_ls = []

        for r in ranks:
            num_ranks = len(cut_df[cut_df['final_rank'] == r])

            for num_rank in range(num_ranks):
                rank_ls.append(last_finish_pos)

            last_finish_pos += num_ranks

        cut_df['final_rank'] = rank_ls

        indices = cut_df.index

        for index in indices:
            result_df['final_rank'][index] = cut_df['final_rank'][index]


        result_df.to_csv('/home/valesco/Datasets/PGA_Data/tournament_results/{}_results.csv'.format(tourny_num),
                        index = False)
    except Exception as e:
        print('Tourny attempt {} failed {}'.format(tourny_num, e))


Tourny attempt 0 failed 'NoneType' object has no attribute 'text'


/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tourny attempt 33 failed invalid literal for int() with base 10: ''
Tourny attempt 64 failed invalid literal for int() with base 10: '$0'
Tourny attempt 94 failed 'NoneType' object has no attribute 'find'
Tourny attempt 200 failed 'NoneType' object has no attribute 'text'
Tourny attempt 203 failed invalid literal for int() with base 10: '-'
Tourny attempt 209 failed invalid literal for int() with base 10: '$0'
Tourny attempt 246 failed 'NoneType' object has no attribute 'text'
Tourny attempt 247 failed 'NoneType' object has no attribute 'text'
Tourny attempt 249 failed 'NoneType' object has no attribute 'text'
Tourny attempt 253 failed 'NoneType' object has no attribute 'text'
Tourny attempt 323 failed 'NoneType' object has no attribute 'text'
Tourny attempt 338 failed 'NoneType' object has no attribute 'text'
Tourny attempt 393 failed invalid literal for int() with base 10: '$0'
Tourny attempt 401 failed 'NoneType' object has no attribute 'find'
Tourny attempt 403 failed 'NoneType' ob

In [99]:
datetime.strftime(start_date, '%m_%d_%Y')

'12_08_2016'

In [81]:
last_finish_pos = result_df['final_rank'].max() + 1

cut_df = result_df.loc[(result_df['final_rank'] == -999)]
cut_df['total_score'] = cut_df['total_score'].apply(lambda x: int(x))
cut_df['final_rank'] = cut_df['total_score'].rank(method = 'dense')
cut_df

/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,player_id,player_name,tournament_name,tournament_start_date,tournament_end_date,num_rounds_played,first_round_score,second_round_score,third_round_score,fourth_round_score,total_score,score_vs_par,final_rank,earnings
72,NaN,Titch Moore,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,72,72,--,--,144,-999.0,1.0,NaN
73,NaN,Lasse Jensen,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,1.0,NaN
74,NaN,Rourke van der Spuy,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,1.0,NaN
75,NaN,Albert Venter (a),Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,1.0,NaN
76,NaN,Jake Redman,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,71,73,--,--,144,-999.0,1.0,NaN
77,NaN,Adilson da Silva,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,71,73,--,--,144,-999.0,1.0,NaN
78,NaN,Oliver Bekker,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,71,73,--,--,144,-999.0,1.0,NaN
79,NaN,Marcus Armitage,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,72,72,--,--,144,-999.0,1.0,NaN
80,NaN,Danie van Tonder,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,69,75,--,--,144,-999.0,1.0,NaN
81,NaN,Arthur Horne,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,1.0,NaN


In [82]:
ranks = cut_df['final_rank'].unique()

rank_ls = []

for r in ranks:
    num_ranks = len(cut_df[cut_df['final_rank'] == r])
       
    for num_rank in range(num_ranks):
        rank_ls.append(last_finish_pos)
        
    last_finish_pos += num_ranks

In [90]:
cut_df['final_rank'] = rank_ls
cut_df

/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,player_id,player_name,tournament_name,tournament_start_date,tournament_end_date,num_rounds_played,first_round_score,second_round_score,third_round_score,fourth_round_score,total_score,score_vs_par,final_rank,earnings,final_ranks
72,NaN,Titch Moore,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,72,72,--,--,144,-999.0,73.0,NaN,73.0
73,NaN,Lasse Jensen,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,73.0,NaN,73.0
74,NaN,Rourke van der Spuy,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,73.0,NaN,73.0
75,NaN,Albert Venter (a),Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,73.0,NaN,73.0
76,NaN,Jake Redman,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,71,73,--,--,144,-999.0,73.0,NaN,73.0
77,NaN,Adilson da Silva,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,71,73,--,--,144,-999.0,73.0,NaN,73.0
78,NaN,Oliver Bekker,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,71,73,--,--,144,-999.0,73.0,NaN,73.0
79,NaN,Marcus Armitage,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,72,72,--,--,144,-999.0,73.0,NaN,73.0
80,NaN,Danie van Tonder,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,69,75,--,--,144,-999.0,73.0,NaN,73.0
81,NaN,Arthur Horne,Alfred Dunhill Championship,2016-12-01,2016-12-04,2.0,74,70,--,--,144,-999.0,73.0,NaN,73.0


In [91]:
indices = cut_df.index

for index in indices:
    result_df['final_rank'][index] = cut_df['final_rank'][index]

/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [92]:
result_df

,player_id,player_name,tournament_name,tournament_start_date,tournament_end_date,num_rounds_played,first_round_score,second_round_score,third_round_score,fourth_round_score,total_score,score_vs_par,final_rank,earnings
0,NaN,Brandon Stone,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,67,66,66,67,266,-22.0,1.0,NaN
1,NaN,Richard Sterne,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,68,70,68,67,273,-15.0,2.0,NaN
2,NaN,Thomas Detry,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,69,71,67,68,275,-13.0,3.0,NaN
3,NaN,Graeme Storm,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,73,67,68,68,276,-12.0,4.0,NaN
4,NaN,Thomas Aiken,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,69,67,71,69,276,-12.0,4.0,NaN
5,NaN,Scott Jamieson,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,67,71,69,69,276,-12.0,4.0,NaN
6,NaN,Carlos Pigem,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,68,69,70,69,276,-12.0,4.0,NaN
7,NaN,Charl Schwartzel,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,66,68,68,74,276,-12.0,4.0,NaN
8,NaN,Benjamin Hebert,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,70,66,68,72,276,-12.0,4.0,NaN
9,NaN,Bryce Easton,Alfred Dunhill Championship,2016-12-01,2016-12-04,4.0,70,69,67,71,277,-11.0,10.0,NaN


In [1]:
'January' in 'asdfaJanuaryasdfasdf'

True